In [180]:
### Hidden Ancestries Script

### HA : (A, taf, x_guess) -> (x_answer, n_iterations, time)

### A generalized function that takes 3 inputs: 
 
 ## 1. Genetic data in a matrix "A" size Nxk containing N SNPs (these are the rows), and k ancestries (these are the columns);

 ## 2. The total allele frequency called "taf" which should be an Nx1 vector
 
 ## 3. A starting guess "x_guess" which should be a kx1 vector
    
### and returns 3 outputs:

 ## 1. The hidden proportions of every ancestry in the data as a kx1 vector
    
 ## 2. The number of iterations that SLSQP did as a scalar value

 ## 3. The run time of the algoirthm as a scalar value, measured in seconds

import numpy as np
import scipy as scipy
from scipy.optimize import minimize
import timeit

def HA(A, taf, x_guess):
    
    # Grab the number of ancestries
    k=np.shape(A)[1]
    
    # Start the clock!
    start = timeit.default_timer()
    
    # This is the objective function!
    def obj_fun(x):
        b=0
        for i in range(0,k):
            b=b + x[i]*A[:,i:(i+1)]
        b=b-taf
        return np.sum(b**2, axis=0)[0]
    
    # This is the gradient of the objective function!
    def grad_obj_fun(x):

        gradvec = np.zeros((k,1))

        d=0

        for i in range(0,k):
            d=d + x[i]*A[:,i:(i+1)]
        d=d-taf

        for i in range(0,k):
            gradvec[i,:] = np.sum(2*A[:,i:(i+1)]*d, axis=0)
        return gradvec

    # These are wrappers that make our constraints and our bounds

    cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x,axis=0) -1},)

    for i in range(0,k-1):
        cons = cons + ({'type': 'ineq', 'fun': lambda x: x[i]},)

    bnds = ((0, None),)

    for i in range(0,k-1):
        bnds = bnds + ((0, None),)

    ans_obj = scipy.optimize.minimize(obj_fun, x_guess, method='SLSQP', jac=grad_obj_fun, bounds=bnds, constraints=cons, tol=1e-5)
    
    stop = timeit.default_timer()
    
    time= stop-start
    
    return ans_obj.x, ans_obj.nit, time

In [181]:
### Initialize data for testing; define matrix A along with strating guess

N=10000 # number of SNPs
j=15 # number of ancestries

A=np.array(np.random.uniform(low=0, high=1, size=(N,1))) # initialize an array for experimental draws

for i in range(1,j):
    A=np.hstack((A,np.random.uniform(low=0, high=1, size=(N,1))))

# First, we choose an answer! This vector must be Nx1

ans=[[0.1], [0.1], [0.1], [0.25], [0.05], [0.1], [0.05], [0.05], [0.01], [0.01], [0.01], [0.01], [0.01], [0.05], [0.1]]

mytaf=A@ans # Total allele frequency

x_t=(1/j)*np.ones((j,1))

In [182]:
what_we_want=HA(A, mytaf, x_t)

print(what_we_want)

print('our correct answer was chosen to be', ans)

(array([0.09998065, 0.09996509, 0.09996001, 0.24987853, 0.04994105,
       0.09981416, 0.04984661, 0.05004293, 0.0098864 , 0.01020446,
       0.01017875, 0.01029378, 0.01024066, 0.04989796, 0.09986895]), 24, 0.1558134864853855)
our correct answer was chosen to be [[0.1], [0.1], [0.1], [0.25], [0.05], [0.1], [0.05], [0.05], [0.01], [0.01], [0.01], [0.01], [0.01], [0.05], [0.1]]
